In [ ]:
import os
while os.getcwd().split('\\')[-1] != 'ecological-networks':
    %cd ..
import setup_paths 
setup_paths.add_path()
from Species_Reintroductions import *
DATA_TYPE = 'Syn_1D'
CASE_TO_SOLVE_init(DATA_TYPE)
get_CASE()

In [ ]:
Log_History = Write_Log(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', '02_Synthetic_Networks_1D', 'Log_Execution.txt'), True)
#Log_Record = Write_Log(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', '02_Synthetic_Networks_1D', 'Log_Record.txt'), True)

In [ ]:
def get_SyntheticData_type01(A = 0.5, C = 0.2, S = 50, seed = 0):
    N = round(S / (1+A))
    M = round((S*A) / (1+A))
    L = round(C*N*M)

    X = np.zeros((N,M),int)
    np.fill_diagonal(X,1)
    
    np.random.seed(seed)
    
    non_filled_rows = np.array(list(itertools.product(np.arange(M,N),np.arange(M))))
    idx = np.array(np.random.choice(len(non_filled_rows),N-M))
    
    for i in range(N-M):
        X[non_filled_rows[idx[i]][0],non_filled_rows[idx[i]][1]] = 1
    
    non_filled_cells = np.argwhere(X==0)
    idx = np.array(np.random.choice(len(non_filled_cells),L - N))
    
    for i in range(L - N):
        X[non_filled_cells[idx[i]][0],non_filled_cells[idx[i]][1]] = 1
    
    if np.all(np.sum(X,axis = 1) > 0) and np.all(np.sum(X,axis = 0) > 0):
        nodf_score = NestednessCalculator(X).nodf(X)
        #print(nodf_score)
        
        M_df = pd.DataFrame(X)
        M_df.columns = np.char.add('B',np.array(range(len(M_df.columns)), dtype = 'str'))
        M_df.index = np.char.add('A',np.array(range(len(M_df.index)), dtype = 'str'))
        
        return M_df, nodf_score
    else:
        return pd.DataFrame(np.zeros((N,M),int)),0.0

def get_SyntheticData_type02(N = 33, M = 17, L = 112, seed = 0):
    X = np.zeros((N,M),int)
    np.fill_diagonal(X,1)
    
    np.random.seed(seed)
    
    non_filled_rows = np.array(list(itertools.product(np.arange(M,N),np.arange(M))))
    idx = np.array(np.random.choice(len(non_filled_rows),N-M))
    
    for i in range(N-M):
        X[non_filled_rows[idx[i]][0],non_filled_rows[idx[i]][1]] = 1
    
    non_filled_cells = np.argwhere(X==0)
    idx = np.array(np.random.choice(len(non_filled_cells),L - N))
    
    for i in range(L - N):
        X[non_filled_cells[idx[i]][0],non_filled_cells[idx[i]][1]] = 1
    
    if np.all(np.sum(X,axis = 1) > 0) and np.all(np.sum(X,axis = 0) > 0):    
        nodf_score = NestednessCalculator(X).nodf(X)
        #print(nodf_score)
        
        M_df = pd.DataFrame(X)
        M_df.columns = np.char.add('B',np.array(range(len(M_df.columns)), dtype = 'str'))
        M_df.index = np.char.add('A',np.array(range(len(M_df.index)), dtype = 'str'))
        
        return M_df,nodf_score
    else:
        return pd.DataFrame(np.zeros((N,M),int)),0.0

In [ ]:
def init_SyntheticData(A = 0.5, C = 0.2, S = 50, ind = 0, FileObj = None):
    M_df_collec = []
    nodf_collec = []
    for seed in range(1000):
        M_df, nodf = get_SyntheticData_type01(A = A, C = C, S = S, seed = seed)
        if nodf != 0:
            M_df_collec.append(M_df)
            nodf_collec.append(nodf)
    del seed,M_df,nodf
    X = pd.DataFrame()
    X['idx'] = np.arange(len(nodf_collec))
    X['nodf'] = nodf_collec
    X = X.sort_values(by = 'nodf')
    
    M_df_selected  = [
        M_df_collec[int(X.iloc[0]['idx'])],
        M_df_collec[int(X.iloc[int(len(M_df_collec)/2)]['idx'])],
        M_df_collec[int(X.iloc[-1]['idx'])]
        ]
    nodf_selected  = [
        nodf_collec[int(X.iloc[0]['idx'])],
        nodf_collec[int(X.iloc[int(len(M_df_collec)/2)]['idx'])],
        nodf_collec[int(X.iloc[-1]['idx'])]
        ]
    if FileObj != None:FileObj.Log_Entry("S = {}, A = {}, C = {}".format(S, A, C))
    for ind_2 in range(3):
        FileName = os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '02_Generate_Database', f"{DATA_TYPE}_Data", f"M_PL_{S}_{ind}_{ind_2}", f"M_PL_{S}_{ind}_{ind_2}-Data_one.pkl")
        os.makedirs(os.path.dirname(FileName), exist_ok=True)
        PickleObj(M_df_selected[ind_2], FileName)
        if FileObj != None:FileObj.Log_Entry(str(nodf_selected[ind_2]) + ' : ' + f"M_PL_{S}_{ind}_{ind_2}")

In [ ]:
S = [100]
A = [0.25, 0.50, 0.75]
C = [0.05, 0.10, 0.15, 0.20]
param_AC = list(itertools.product(S,A,C))
for ind in range(len(param_AC)):
    s,a,c = param_AC[ind]
    #print(a,c)
    #Log_Record.Log_Entry("{} : (S,A,C) = ({},{},{})".format(ind,s,a,c))
    try:
        init_SyntheticData(A = a, C = c, S = s, ind = ind, FileObj = Log_History)
    except:
        pass

In [ ]:
Data_Preparation(
    param_Segment01 = None,
    param_Segment02 = [
        10,                    ## param_Ensembles
        np.arange(9),           ## param_Fraction
        0,                      ## Start
        None,                   ## End
        1,                      ## Step
        True,                   ## Overwrite 
        False,                  ## Display_Output
        False                   ## Display_Count
        ], 
    File_Obj = Log_History, 
    Record_Obj = None,
    Record_Reset = False
    );

In [ ]:
M = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', f"M_{DATA_TYPE.split('_')[0]}.pkl"))
WebOfLife_ID = list(UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', f"Network_Solved_{DATA_TYPE.split('_')[0]}.pkl")).NetworkName)

In [ ]:
for NetworkName in itertools.islice(WebOfLife_ID,31,32,1):
    Solve_For_Network(
        NetworkName = NetworkName,                  ## Network Name
        Ensembles_batch = [0,10],                   ## Mini-Batch of Ensembles
        Keys_Filter = [[0,1,2], [2,5,8], [0,2]],  ## Filter: Strategy, Fraction, Target
        Flag = [True, True, True, True, True],      ## System, Degree, Closeness, Betweenness
        Overwrite = True,                           ## Overwrite existing results
        File_Obj = Log_History                      ## History of Operations
        )

In [ ]:
M = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', f"M_{DATA_TYPE.split('_')[0]}.pkl"))
WebOfLife_ID = list(UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', f"Network_Solved_{DATA_TYPE.split('_')[0]}.pkl")).NetworkName)

Keys_Filter = [[0,1,2], [2,5,8], [0,2]]
Ensembles_batch = [0,10]

Flag = [True, True, True, True, True]

## Done :- ind 0,29 (Left: 30,31)
# 0:15
# 0:20
# 20:30
if Log_History != None:
    Log_History.Log_Entry('.......Solution Averaging.........')
    Log_History.Log_Entry("Key Filter: {}".format(Keys_Filter))
    Log_History.Log_Entry("Ensembles Run: {}".format(Ensembles_batch))

In [ ]:
for NetworkName in itertools.islice(WebOfLife_ID,0,None,1):
    M_df = M[NetworkName]
    data_two = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '02_Generate_Database', f"{DATA_TYPE}_Data", f"{NetworkName}", f"{NetworkName}-Data_two.pkl"))
    data_two = {k: v[Ensembles_batch[0]:min(Ensembles_batch[1],len(v))] for k,v in data_two.items() if k in set(list(itertools.product(*Keys_Filter)))}
    
    if Log_History != None:
        Log_History.Log_Entry('Network ID: ' + NetworkName)
        Log_History.Log_Entry("Shape: {} ({})".format(M_df.shape, sum(M_df.shape)))
        
    if Flag[0]:
        try:
            data_three = getDict_x(
                ID = 'Data_three', 
                data_two = data_two, 
                NetworkName = NetworkName, 
                Shape = M_df.shape, 
                Steady_State_Correction = True
                );
            if Log_History != None:
                Log_History.Log_Entry("{} ; {}".format(NetworkName, 'System_Driven'))
        except:
            if Log_History != None:
                Log_History.Log_Entry("FAILED!: {} ; {}".format(NetworkName, 'System_Driven'))

    if Flag[1]:
        try:
            data_four = getDict_x(
                ID = 'Data_four', 
                data_two = data_two, 
                NetworkName = NetworkName, 
                Shape = M_df.shape, 
                Steady_State_Correction = True
                );
            if Log_History != None:
                Log_History.Log_Entry("{} ; {}".format(NetworkName, 'Degree_Driven'))
        except:
            if Log_History != None:
                Log_History.Log_Entry("FAILED!: {} ; {}".format(NetworkName, 'Degree_Driven'))

    if Flag[2]:
        try:
            data_five = getDict_x(
                ID = 'Data_five', 
                data_two = data_two, 
                NetworkName = NetworkName, 
                Shape = M_df.shape,
                Steady_State_Correction = True
                );
            if Log_History != None:
                Log_History.Log_Entry("{} ; {}".format(NetworkName, 'Closeness_Driven'))
        except:
            if Log_History != None:
                Log_History.Log_Entry("FAILED!: {} ; {}".format(NetworkName, 'Closeness_Driven'))

    if Flag[3]:
        try:
            data_six = getDict_x(
                ID = 'Data_six', 
                data_two = data_two, 
                NetworkName = NetworkName, 
                Shape = M_df.shape, 
                Steady_State_Correction = True
                );
            if Log_History != None:
                Log_History.Log_Entry("{} ; {}".format(NetworkName, 'Betweenness_Driven'))
        except:
            if Log_History != None:
                Log_History.Log_Entry("FAILED!: {} ; {}".format(NetworkName, 'Betweenness_Driven'))
            
    if Flag[4]:
        try:
            data_rand = getDict_x(
                ID = 'Data_rand', 
                data_two = data_two, 
                NetworkName = NetworkName, 
                Shape = M_df.shape, 
                Steady_State_Correction = True
                );
            if Log_History != None:
                Log_History.Log_Entry("{} ; {}".format(NetworkName, 'Random'))
        except:
            if Log_History != None:
                Log_History.Log_Entry("FAILED!: {} ; {}".format(NetworkName, 'Random'))
    print(NetworkName)

In [ ]:
try:
    Log_History.close_Log()
except:
    pass
try:
    Log_Record.close_Log()
except:
    pass